# Example for generating photoionization models

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pyCloudy as pc

from sys import path
path.insert(0,'../CloudyGalaxy/')

from gas_stats import GasStatsBy17 as GasStats
from abundances import ElementalAbundances, convert_xi_to_F
from read_write_cloudy import StellarSpectrum, format_emission_table, make_input_file, make_emission_line_files

### Setting the data paths and cores to use for Cloudy

In [15]:
n_proc              = 6

output_dir          = './output_data/'
model_name          = 'test_model_low_res_age_2Myr_new'
data_path           = '../CloudyGalaxy/input_data/'
star_table_filename = 'Chabrier_constant_SFH_IMF_lower_0p08_IMF_upper_120_star_table.ASCII'
emission_line_list  = format_emission_table('../CloudyGalaxy/input_data/LineList_HII.dat')


### Defining the grid of free parameters for which to produce models

In [17]:
nlogZs = 8
nlogUs = 4
nxis   = 4
ntaus  = 6

logZs = np.linspace(-1.0, 0.7, nlogZs)
logUs = np.linspace(-4.0, -1.0, nlogUs)
xis   = np.linspace(0.1, 0.6, nxis)
taus  = np.linspace(0.01, 4.0, ntaus)
Fs    = np.ones((nlogZs,nxis))*np.nan


### Making the Cloudy input files

In [19]:
pc.print_make_file(dir_ = output_dir)
for i in range(nlogZs):
    logZ = logZs[i]
    xi_to_F = convert_xi_to_F(logZ)
    for j in range(nlogUs):
        for k in range(nxis):
            logU = logUs[j]
            xi = xis[k]
            F = xi_to_F(xi)
            Fs[i,k] = F
            stellar_spectrum = StellarSpectrum(data_path, star_table_filename)
            gas_stats = GasStats(stellar_spectrum, logZ, logU, r_inner=1e19, age=2e6)
            gas_stats.calc_all_parameters()
            print(gas_stats.Q)
            print(gas_stats.Q_hat)
            print(gas_stats.M_star)
            elemental_abundances = ElementalAbundances(logZ, F)
            elemental_abundances.calc_all_parameters()
            make_input_file(output_dir, model_name, logZ, logU, xi, emission_line_list, stellar_spectrum, gas_stats, elemental_abundances, age=2e6)


3.767303134617706e+47
5.634255848813399e+46
6.686425387322653
3.767303134617706e+47
5.634255848813399e+46
6.686425387322653
3.767303134617706e+47
5.634255848813399e+46
6.686425387322653
3.767303134617706e+47
5.634255848813399e+46
6.686425387322653
3.7673031346177065e+48
5.634255848813399e+46
66.86425387322655
3.7673031346177065e+48
5.634255848813399e+46
66.86425387322655
3.7673031346177065e+48
5.634255848813399e+46
66.86425387322655
3.7673031346177065e+48
5.634255848813399e+46
66.86425387322655
3.767303134617707e+49
5.634255848813399e+46
668.6425387322656
3.767303134617707e+49
5.634255848813399e+46
668.6425387322656
3.767303134617707e+49
5.634255848813399e+46
668.6425387322656
3.767303134617707e+49
5.634255848813399e+46
668.6425387322656
3.767303134617707e+50
5.634255848813399e+46
6686.425387322655
3.767303134617707e+50
5.634255848813399e+46
6686.425387322655
3.767303134617707e+50
5.634255848813399e+46
6686.425387322655
3.767303134617707e+50
5.634255848813399e+46
6686.425387322655
3.76

### Running Cloudy

In [20]:
pc.run_cloudy(dir_ = output_dir, n_proc = n_proc, model_name = model_name, use_make = True)

### Extracting the emission line luminosities from the Cloudy data files and storing them in .npy files

In [22]:
make_emission_line_files(output_dir, model_name, logZs, logUs, xis, taus, Fs)

./output_data/test_model_low_res_age_2Myr_new_-1.000e+00_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-1.000e+00_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.2667, tau = 2.4040.
PROCESSING: logZ = -1.0000, logU = -4.0000, xi = 0.2667, tau = 3.2020.
PROCESSING: logZ = -1.0000, log

PROCESSING: logZ = -1.0000, logU = -1.0000, xi = 0.6000, tau = 3.2020.
PROCESSING: logZ = -1.0000, logU = -1.0000, xi = 0.6000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-7.571e-01_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-7.571e-01_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = -0.7571, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = -0.7571, log

PROCESSING: logZ = -0.7571, logU = -1.0000, xi = 0.6000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-5.143e-01_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-5.143e-01_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = -0.5143, logU = -4.0000, xi = 0.2667, tau = 2.4040.
PROCESSING: logZ = -0.5143, log

1 models found!
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-2.714e-01_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.2667, tau = 2.4040.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.2667, tau = 3.2020.
PROCESSING: logZ = -0.2714, logU = -4.0000, xi = 0.2667, tau = 4.0000.
./output_data/test_model_low_res_age_2

./output_data/test_model_low_res_age_2Myr_new_-2.857e-02_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_-2.857e-02_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.2667, tau = 2.4040.
PROCESSING: logZ = -0.0286, logU = -4.0000, xi = 0.2667, tau = 3.2020.
PROCESSING: logZ = -0.0286, log

PROCESSING: logZ = -0.0286, logU = -1.0000, xi = 0.6000, tau = 3.2020.
PROCESSING: logZ = -0.0286, logU = -1.0000, xi = 0.6000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_2.143e-01_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_2.143e-01_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = 0.2143, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = 0.2143, logU = -4.0000,

PROCESSING: logZ = 0.2143, logU = -1.0000, xi = 0.6000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_4.571e-01_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_4.571e-01_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = 0.4571, logU = -4.0000, xi = 0.2667, tau = 2.4040.
PROCESSING: logZ = 0.4571, logU = -4.0000, x

PROCESSING: logZ = 0.4571, logU = -1.0000, xi = 0.6000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_7.000e-01_-4.000e+00_1.000e-01
1 models found!
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.1000, tau = 0.0100.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.1000, tau = 0.8080.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.1000, tau = 1.6060.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.1000, tau = 2.4040.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.1000, tau = 3.2020.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.1000, tau = 4.0000.
./output_data/test_model_low_res_age_2Myr_new_7.000e-01_-4.000e+00_2.667e-01
1 models found!
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.2667, tau = 0.0100.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.2667, tau = 0.8080.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.2667, tau = 1.6060.
PROCESSING: logZ = 0.7000, logU = -4.0000, xi = 0.2667, tau = 2.4040.
PROCESSING: logZ = 0.7000, logU = -4.0000, x